<a href="https://www.kaggle.com/code/yaramahrous/backpropagation?scriptVersionId=198107863" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.optimizers import Adam, SGD

In [ ]:
class NeuralNetwork:
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size=1):
        self.input_size = input_size
        self.hidden_size1 = hidden_size1
        self.hidden_size2 = hidden_size2 
        self.output_size = output_size
        
        self.weights_input_hidden1 = np.random.randn(self.input_size, self.hidden_size1)
        self.bias_hidden1 = np.zeros((1, self.hidden_size1))
        
        self.weights_hidden1_hidden2 = np.random.randn(self.hidden_size1, self.hidden_size2)
        self.bias_hidden2 = np.zeros((1, self.hidden_size2))
        
        self.weights_hidden2_output = np.random.randn(self.hidden_size2, self.output_size)
        self.bias_output = np.zeros((1, self.output_size))
        
    def count_parameters(self):
        input_hidden1_params = self.input_size * self.hidden_size1 + self.hidden_size1
        hidden1_hidden2_params = self.hidden_size1 * self.hidden_size2 + self.hidden_size2 
        hidden2_output_params = self.hidden_size2 * self.output_size + self.output_size 
        
        total_params = input_hidden1_params + hidden1_hidden2_params + hidden2_output_params
        return total_params
        
    def tanh (self , x):
        return np.tanh(x)
    
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def tanh_derivative(self,x):
        return (1 / np.cosh(x))**2
    
    def sigmoid_derivative(self, x):
        return x * (1 - x)
    
    def forward(self, x):
        # First hidden layer
        self.hidden_activation1 = np.dot(x, self.weights_input_hidden1) + self.bias_hidden1
        self.hidden_output1 = self.tanh(self.hidden_activation1)

        # Second hidden layer
        self.hidden_activation2 = np.dot(self.hidden_output1, self.weights_hidden1_hidden2) + self.bias_hidden2
        self.hidden_output2 = self.tanh(self.hidden_activation2)
        
        # Output layer
        self.output_activation = np.dot(self.hidden_output2, self.weights_hidden2_output) + self.bias_output
        self.predicted_output = self.sigmoid(self.output_activation)

        return self.predicted_output
    
    def backward(self, x, y, learning_rate):
        output_error = y - self.predicted_output
        output_delta = output_error * self.sigmoid_derivative(self.predicted_output)

        hidden_error2 = np.dot(output_delta, self.weights_hidden2_output.T)
        hidden_delta2 = hidden_error2 * self.tanh_derivative(self.hidden_output2)

        hidden_error1 = np.dot(hidden_delta2, self.weights_hidden1_hidden2.T)
        hidden_delta1 = hidden_error1 * self.tanh_derivative(self.hidden_output1)
        
        self.weights_hidden2_output += np.dot(self.hidden_output2.T, output_delta) * learning_rate
        self.bias_output += np.sum(output_delta, axis=0, keepdims=True) * learning_rate
        
        self.weights_hidden1_hidden2 += np.dot(self.hidden_output1.T, hidden_delta2) * learning_rate
        self.bias_hidden2 += np.sum(hidden_delta2, axis=0, keepdims=True) * learning_rate
        
        self.weights_input_hidden1 += np.dot(x.T, hidden_delta1) * learning_rate
        self.bias_hidden1 += np.sum(hidden_delta1, axis=0, keepdims=True) * learning_rate
    
        
    def train(self, x, y, epochs, learning_rate):
        for epoch in range(1,epochs+1):
            output = self.forward(x)
            self.backward(x, y, learning_rate)
            loss = np.mean(np.square(y - output))
            print(f'Epoch: {epoch}/{epochs}, Accuracy:{1-loss}, Loss: {loss}')
    
    def predict(self,x):
        return self.forward(x)

In [ ]:
df = pd.read_csv('/kaggle/input/healthcare-diabetes/Healthcare-Diabetes.csv')
df.head()

In [ ]:
df.describe()

In [ ]:
df.isna().sum()

In [ ]:
df.drop(columns=['Id'],axis=1,inplace=True)

In [ ]:
x = df.drop(columns='Outcome')
y = df['Outcome']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
y_train.value_counts()

In [ ]:
smote = SMOTE(random_state=42)
x_train,y_train = smote.fit_resample(x_train,y_train)

In [ ]:
y_train.value_counts()

In [ ]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
model = Sequential(
    [
        Input(shape=(x_train.shape[1],)),
        Dense(10, activation='tanh'),
        Dense(5, activation='tanh'),
        Dense(1, activation='sigmoid')
    ]
)

model.summary()

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.025), loss='binary_crossentropy', metrics=['binary_accuracy'])

In [ ]:
model.fit(x=x_train, y=y_train, batch_size=20, epochs=30, shuffle=True, verbose=2,validation_split=0.1)

In [ ]:
preds = model.predict(x_test)
pred_labels = (preds > 0.5).astype(int)

print(classification_report(y_test, pred_labels))

# Now lets try our NN :)

In [ ]:
nn = NeuralNetwork(input_size=x_train.shape[1], hidden_size1=10, hidden_size2=5, output_size=1)
nn.count_parameters()


In [ ]:
y_train_array = y_train.values.reshape(-1, 1)
y_train_array


In [ ]:
nn.train(x_train,y_train_array,epochs=30,learning_rate = 0.025)

In [ ]:
preds = nn.predict(x_test)
pred_labels = (preds > 0.5).astype(int)

print(classification_report(y_test, pred_labels))